## Start up
- First upload the ifc mode, then the pointcloud or gaussian splatted file. Make sure to upload the Gaussian splatted file as normal .ply file, not as gaussian splatted file.
- Open *Scripting* in the topbar and press *New*.
- In the topbar, go to *Windows* and click *Toggle system console*. If you print something in the scripting, it will be shown here.

## Make vertex groups
Paste the below code in the Scripting window. 

In [ ]:
# RUN 2 TIMES
# TO DO: Adjust first_cluster and last_cluster to the clusternumbers (starting and ending number)
# TO DO: Adjust column_name to the name of the column that shows the cluster numbers
import bpy
first_cluster = 0
last_cluster = 6
column_name = 'cluster'

obj = bpy.context.active_object
bpy.ops.object.mode_set(mode='OBJECT')
cluster_data = obj.data.attributes[column_name].data

for cluster_value in range(first_cluster , last_cluster+1): 
    vg = obj.vertex_groups.get(f"Cluster {cluster_value}")
    if not vg:
        vg = obj.vertex_groups.new(name=f"Cluster {cluster_value}")
    vertices_to_assign = [i for i, cluster in enumerate(cluster_data) if cluster.value == cluster_value]
    vg.add(vertices_to_assign, 1.0, 'ADD') 
bpy.ops.object.mode_set(mode='EDIT')
print('Done')

You can see the vertex groups here. Click on a cluster and press *Select* to see which points belong to this cluster.


![title](../../assets/see%20vertex%20groups.png)

There are two methods to check if a point is inside a building part. The first one shows how to do check it when taking the mesh as bounding box. The second one does it by calculating it by calcualting if the point lies inside the mesh.

The following code checks how many points are in the bounding box of two building parts.

In [ ]:
import bpy
import csv
from mathutils import Vector

# TO DO: Define file paths and names correctly
point_cloud_name = 'file_name_inBlender' # name of clutered point cloud/ gaussian spat file in blender
csv_output_path = r"../../7. Results/Accuracy csv scores"
csv_name = "output.csv"

# TO DO:
# Define here the meshes where you want to check how many points are in that bounding box.
# In this case there are multiple building parts that consist of one or multiple files in blender. 
# Make sure the items in the dictionary are the names of the building parts in blender
building_parts = {
    'canopy': ['IfcBuildingElementProxy/Model Text:200:219863', 'IfcBuildingElementProxy/sign:sign:217541'],
    'door': ['IfcBuildingElementProxy/Door header:Door header:223134', 'IfcDoor/双面嵌板镶玻璃门 3 - 带亮窗:1800 x 2600mm:22', 'IfcDoor/常规模型 2:常规模型 2:231152', 'IfcWallStandardCase/Basic Wall:exterior wall:205165'],
    'window': ['IfcWindow/窗 1:窗 1:209621'],
    'stairs': ['IfcStair/常规模型 3:常规模型 3:232031', 'IfcStair/常规模型 3:常规模型 3:232031.001'],
    'wall': ['IfcWall/墙 1:墙 1:206041', 'IfcWall/墙 2:墙 2:208178'],
    'roof': ['IfcRoof/常规模型 4:常规模型 4:233826']
}


csv_filename = csv_output_path + "\\" + csv_name  
point_cloud = bpy.data.objects[point_cloud_name]
vertex_groups = point_cloud.vertex_groups


with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file) 
    writer.writerow(["Cluster", "Building Part", "Points", "Percentage"])  

    for cluster_group in vertex_groups:
        print(f"\nCluster: {cluster_group.name}")
        total_cluster_points = 0
        points_in_building_parts = {key: 0 for key in building_parts}
        
        for vertex in point_cloud.data.vertices:
            for group in vertex.groups:
                if group.group == cluster_group.index:
                    total_cluster_points += 1
                    vertex_location = point_cloud.matrix_world @ vertex.co
                    for part_key, part_objects in building_parts.items():
                        for obj_name in part_objects:
                            ifc_obj = bpy.data.objects.get(obj_name)
                            if ifc_obj:
                                world_coords = [ifc_obj.matrix_world @ Vector(corner) for corner in ifc_obj.bound_box]
                                if (world_coords[0][0] <= vertex_location[0] <= world_coords[6][0] and
                                    world_coords[0][1] <= vertex_location[1] <= world_coords[6][1] and
                                    world_coords[0][2] <= vertex_location[2] <= world_coords[6][2]):
                                    points_in_building_parts[part_key] += 1
                                    break
        print(f"Total points in cluster: {total_cluster_points}")
        writer.writerow([cluster_group.name, 'TOTAL', total_cluster_points, 100])
        for part_key, count in points_in_building_parts.items():
            percentage = round(count / total_cluster_points * 100, 2) if total_cluster_points > 0 else 0
            print(f"Points in cluster matching {part_key}: {count} -> {percentage}%")
            writer.writerow([cluster_group.name, part_key, count, percentage])
        remaining = total_cluster_points - sum(points_in_building_parts.values())
        remaining_percentage = round(remaining / total_cluster_points * 100, 2) if total_cluster_points > 0 else 0
        writer.writerow([cluster_group.name, 'Remaining', remaining, remaining_percentage])

check if a point is inside the mesh

In [ ]:
import bpy
import csv

# TO DO: Define file paths and names correctly
point_cloud_name = 'file_name_inBlender' # name of clutered point cloud/ gaussian spat file in blender
csv_output_path = r"../../7. Results/Accuracy csv scores"
csv_name = "output.csv"

# TO DO:
# Define here the meshes where you want to check how many points are in that bounding box.
# In this case there are multiple building parts that consist of one or multiple files in blender. 
# Make sure the items in the dictionary are the names of the building parts in blender
building_parts = {
    'canopy': ['IfcBuildingElementProxy/Model Text:200:219863', 'IfcBuildingElementProxy/sign:sign:217541'],
    'door': ['IfcBuildingElementProxy/Door header:Door header:223134', 'IfcDoor/双面嵌板镶玻璃门 3 - 带亮窗:1800 x 2600mm:22', 'IfcDoor/常规模型 2:常规模型 2:231152', 'IfcWallStandardCase/Basic Wall:exterior wall:205165'],
    'window': ['IfcWindow/窗 1:窗 1:209621'],
    'stairs': ['IfcStair/常规模型 3:常规模型 3:232031', 'IfcStair/常规模型 3:常规模型 3:232031.001'],
    'wall': ['IfcWall/墙 1:墙 1:206041', 'IfcWall/墙 2:墙 2:208178'],
    'roof': ['IfcRoof/常规模型 4:常规模型 4:233826']
}

def is_inside(point, obj):    
    _point = point - obj.location
    _, closest, nor, _ = obj.closest_point_on_mesh(_point, )

    direction = closest - _point

    if direction.dot(nor) > 0:
        return True
    else:
        return False

csv_filename = csv_output_path + "\\" + csv_name  
point_cloud = bpy.data.objects[point_cloud_name]
vertex_groups = point_cloud.vertex_groups

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Cluster", "Building Part", "Points", "Percentage"])

    # Evaluate each cluster
    for cluster_group in vertex_groups:
        print(f"\nCluster: {cluster_group.name}")
        total_cluster_points = 0
        points_in_building_parts = {key: 0 for key in building_parts}

        # Iterate over vertices
        for indx, vertex in enumerate(point_cloud.data.vertices_:
            for group in vertex.groups:
                if group.group == cluster_group.index:
                    total_cluster_points += 1
                    vertex_location = point_cloud.matrix_world @ vertex.co
                    for part_key, part_objects in building_parts.items():
                        for obj_name in part_objects:
                            ifc_obj = bpy.data.objects.get(obj_name)
                            depsgraph = bpy.context.evaluated_depsgraph_get()
                            eval_obj = ifc_obj.evaluated_get(depsgraph)
                            if ifc_obj and is_inside(vertex_location, eval_obj):
                                points_in_building_parts[part_key] += 1
                                if part_key ==
                                break

        print(f"Total points in cluster: {total_cluster_points}")
        writer.writerow([cluster_group.name, 'TOTAL', total_cluster_points, 100])

        for part_key, count in points_in_building_parts.items():
            percentage = round(count / total_cluster_points * 100, 2) if total_cluster_points > 0 else 0
            print(f"Points in cluster matching {part_key}: {count} -> {percentage}%")
            writer.writerow([cluster_group.name, part_key, count, percentage])

        remaining = total_cluster_points - sum(points_in_building_parts.values())
        remaining_percentage = round(remaining / total_cluster_points * 100, 2) if total_cluster_points > 0 else 0
        writer.writerow([cluster_group.name, 'Remaining', remaining, remaining_percentage])

print("Done")